## Selenium 을 이용한 동적 크롤링

In [16]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

from urllib.request import urlopen
import pandas as pd
import bs4
import requests

### 실행 중에 JS통해 동적으로 변환된 값 추출 
- 처음에는 '댓글' 이라고 써있었는데 자바스크립트가 동적으로 댓글 수에 따라 숫자 변화될때
- 정적 페이지에서는 추출 안됨
- 셀레니움 사용해서 동적페이지를 별도로 열어서 추출

In [17]:
url =  'https://n.news.naver.com/mnews/article/079/0003812365?sid=100'
html = urlopen(url)
bs_obj = bs4.BeautifulSoup(html, 'html.parser')


In [3]:
# 제목 : 추출 가능
title = bs_obj.find('div', {'class' : 'media_end_head_title'}).select_one('span').text
title

'尹 "가짜뉴스, AI 이용해 빠른 속도 확산…자유민주주의 훼손, 미래 망쳐"'

In [4]:
# 좋아요 수 추출
like_num = bs_obj.find('span', {'class':'u_likeit_text _count num'})
like_num
if like_num is None:
    print('None')

None


In [5]:
# comment_count수 추출
comment_count = bs_obj.find('a', {'id':'comment_count'}).text
comment_count
if comment_count is None:
    print('None')

'댓글'

In [6]:
###############################

#### Selenium 패키지 모듈 이용한 자동 크롤링
- selenium
- webdriver 라는 API를 통해 운영체제에 설치된 웹 브라우저를 제어하는 함수를 포함한 패키지
- Thirdparty library이기 때문에 설치해줘야한다.
- 
- Beautiful Soup과 함께 사용할 수 있어 훨씬 쉽게 크롤링할 수 있음
- requests를 통한 .text의 경우 브라우저에서 ‘소스보기’를 한 것과 같이 동작 (정적 페이지)
- 자바스크립트를 통해 동적으로 변환된 DOM 요소에 접근 불가
- 반면 Selenium은 실제 웹 브라우저가 동작하기 때문에
- 자바스크립트 실행이 완료된 후에 동적으로 변환된 DOM 결과물에 접근 가능


In [22]:
# selenium 메소드
# driver.find_element(By.CLASS_NAME, "information")
# driver.find_element(By.CSS_SELECTOR, "#fname")
# driver.find_element(By.ID, "lname")
# driver.find_element(By.LINK_TEXT, "Selenium Official Page")
# driver.find_element(By.NAME, "newsletter")
# driver.find_element(By.PARTIAL_LINK_TEXT, "Official Page")
# driver.find_element(By.TAG_NAME, "a")
# driver.find_element(By.XPATH, "//input[@value='f']")
# driver가 접근한 페이지의 source 추출

In [7]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


<!-- driver.find_element(By.CLASS_NAME, "information")
driver.find_element(By.CSS_SELECTOR, "#fname")
driver.find_element(By.ID, "lname")
driver.find_element(By.LINK_TEXT, "Selenium Official Page")
driver.find_element(By.NAME, "newsletter")
driver.find_element(By.PARTIAL_LINK_TEXT, "Official Page")
driver.find_element(By.TAG_NAME, "a")
driver.find_element(By.XPATH, "//input[@value='f']")
driver가 접근한 페이지의 source 추출
html = driver.page_source -->

In [8]:
# ver 4에서만 driver관련 함수 사용하기 때문에 버전 체크 필수

In [9]:
pip show selenium

Name: selenium
Version: 4.12.0
Summary: 
Home-page: https://www.selenium.dev
Author: 
Author-email: 
License: Apache 2.0
Location: /opt/homebrew/Cellar/jupyterlab/4.0.5_1/libexec/lib/python3.11/site-packages
Requires: certifi, trio, trio-websocket, urllib3
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install webdriver_manager

Note: you may need to restart the kernel to use updated packages.


In [1]:
# 주의 !
# driver를 통해서 연 브라우저는 driver.close()해서 닫음
# 다시 (1)webdriver객체 생성을 할 때 오류가 발생하면
# (1) Kernal / Restart kernal
# 그래도 오류나면 주피터 재시작

In [9]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By   # 셀레니움 4.0 부터 포함된 객체(모듈)

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [10]:
# (1) webdriver객체 생성
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

In [11]:
# 페이지 접속 - 여기서 접속된(자동으로 열린) 페이지에서 크롤링이된다!
# 열려있을때만 작동하니 driver를 통해 닫아야함!!!!! 먼저 닫지 말기
driver.get('https://n.news.naver.com/mnews/article/079/0003812365?sid=100')

In [12]:
# 좋아요 수 추출
# 클래스 선택자로 선택
like_num = driver.find_element(By.CSS_SELECTOR, '.u_likeit_text._count.num')
like_num.text

# 클래스 이름으로 선택
like_num = driver.find_element(By.CLASS_NAME, 'u_likeit_text._count.num')
like_num.text
# 선택자 주의!
# 중간에 스페이스 있으면 안됨 : (.)점으로 연결해야 함

'2'

'2'

In [14]:
# 댓글 수 추출 : 'a',{'id':'comment_count'}
# 하나찾아오는 거는 find_element
comment_count = driver.find_element(By.ID, 'comment_count')
comment_count.text

'36'

In [21]:
# 정적 데이터도 셀레니움 사용 가능
paper = driver.find_element(By.CSS_SELECTOR, '.media_end_head_top_logo img')
paper.get_attribute('title')     # <img> 태그의 title속성값 추출

title = driver.find_element(By.CSS_SELECTOR, '.media_end_head_title').text
title


datetime = driver.find_element(By.CSS_SELECTOR, '.media_end_head_info_datestamp_time._ARTICLE_DATE_TIME').text
datetime

'노컷뉴스'

'尹 "가짜뉴스, AI 이용해 빠른 속도 확산…자유민주주의 훼손, 미래 망쳐"'

'2023.09.13. 오후 3:07'

In [24]:
# 동일한 내용 정적 크롤링으로 데이터 추출
paper = bs_obj.find('a', {'class':'media_end_head_top_logo'}).select_one('img')['title']
paper

title = bs_obj.find('div', {'class' : 'media_end_head_title'}).select_one('span').text
title

datetime = bs_obj.find('span', {'class' : 'media_end_head_info_datestamp_time _ARTICLE_DATE_TIME'}).text
datetime

'노컷뉴스'

'尹 "가짜뉴스, AI 이용해 빠른 속도 확산…자유민주주의 훼손, 미래 망쳐"'

'2023.09.13. 오후 3:07'

In [27]:
# find_elements() : 여러 객체를 리스트로 반환
dates = driver.find_elements(By.CSS_SELECTOR, '.media_end_head_info_datestamp div')
dates  # 셀레니움 객체

for d in dates:
    print(d.text)

[<selenium.webdriver.remote.webelement.WebElement (session="9bf9cf4ef9b762bf3ad6e4ef370d139f", element="1CDBCB6A737B7AC1FD6EF35AA4FF1B77_element_51")>]

입력2023.09.13. 오후 3:07


In [28]:
driver.close()